## Structured Output

"Labrador is a breed of Dog"

{
    'content':'Dog',
    'species':'Labra'
}

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from pydantic import BaseModel
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')


In [3]:
key = "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"


In [4]:
from pydantic import BaseModel

class PlanetInfo(BaseModel):
    name: str
    color: str
    distance_miles: int

In [5]:
open_router_model_client =  OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-chat-v3-0324:free",
    api_key = key,
    model_info={
        "family":'deepseek',
        "vision" :True,
        "function_calling":True,
        "json_output": False
    }
)

c:\Users\shubu\Desktop\AutoGen Hardcode\venv\lib\site-packages\autogen_ext\models\openai\_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [6]:
agent = AssistantAgent(
    name='planet_agent',
    model_client=open_router_model_client,
    system_message="You are a helpful assistant that provides information about planets. in the structure JSON" \
    "{ name :str" \
    "age : int" \
    "}",
    output_content_type = PlanetInfo
)

In [9]:
async def test_structured_output():
    task = TextMessage(content = "Please provide information about Mars.",source='user')
    result = await agent.run(task=task)
    structured_response = (result.messages[-1].content)
    print(structured_response)

await test_structured_output()

KeyError: 'structured_output'

In [8]:
from typing import Literal
from autogen_agentchat.ui import Console
from pydantic import BaseModel


# The response format for the agent as a Pydantic base model.
class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]


# Create an agent that uses the OpenAI GPT-4o model.
agent = AssistantAgent(
    "assistant",
    model_client=open_router_model_client,
    system_message="Categorize the input as happy, sad, or neutral following the JSON format.",
    # Define the output content type of the agent.
    output_content_type=AgentResponse,
)

result = await Console(agent.run_stream(task="I am happy."))

# Check the last message in the result, validate its type, and print the thoughts and response.
assert isinstance(result.messages[-1], StructuredMessage)
assert isinstance(result.messages[-1].content, AgentResponse)
print("Thought: ", result.messages[-1].content.thoughts)
print("Response: ", result.messages[-1].content.response)
await model_client.close()


---------- TextMessage (user) ----------
I am happy.


KeyError: 'structured_output'